In [11]:
from gensim import corpora, models, similarities
import gensim

In [12]:
import json 
import gzip 
def parse(path): 
    g = gzip.open(path, 'r') 
    for l in g: 
        yield eval(l)

import gensim
class MyReviews():


    def __iter__(self):

        review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
        #for review_item in parse("../../download/reviews_Cell_Phones_and_Accessories.json.gz"): 
        for it in range(100000):
            if it%10000==0:
                print("it="+str(it))
            review_item = next(review_item_iterator)
            #print(review_item)
        #for review_item in self.reviews_cursor:


            reviewText = review_item["reviewText"]
            label = review_item["asin"] + "_" + review_item["reviewerID"] 
            preprocess_reviewText = gensim.utils.simple_preprocess(reviewText)
                  

            #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
            yield preprocess_reviewText

In [26]:
import gensim
def myReviews(catFilter = None, asin = None, concatenate = False):
    preprocess_reviewText = []
    review_item_iterator = iter(parse("../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.json.gz"))
    it = 0
    conc_count = 0
    for review_item in review_item_iterator:
        it += 1
    #for it in range(1000000):
        if it%100000==0:
            print("it="+str(it))

        reviewText = review_item["reviewText"]
        label = review_item["asin"] + "_" + review_item["reviewerID"] 
        preprocess_reviewText_i = gensim.utils.simple_preprocess(reviewText)


        #yield gensim.models.doc2vec.LabeledSentence(words=preprocess_reviewText, tags=[label])
        if (catFilter == None or (catFilter in review_item["categories"])) and (asin == None or (asin == review_item["asin"])):
            if concatenate==False:
                yield preprocess_reviewText_i
            else:
                conc_count += 1
                preprocess_reviewText.extend(preprocess_reviewText_i)
                print("Concatenated count: "+str(conc_count))
    if concatenate == True:
        print("Return the final text with length: "+str(len(preprocess_reviewText)))
        print(preprocess_reviewText)
        yield preprocess_reviewText

In [14]:
import os
dictionaryFileName = '../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat.dict'
if os.path.isfile(dictionaryFileName)==False:
    dictionary = corpora.Dictionary(myReviews("Cell Phones"))
    from six import iteritems
    once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
    dictionary.filter_tokens(once_ids)  # remove stop words and words that appear only once
    dictionary.compactify()  # remove gaps in id sequence after words that were removed
    dictionary.save(dictionaryFileName)  # store the dictionary, for future reference
else:
    dictionary = corpora.Dictionary.load(dictionaryFileName)

print(dictionary)

Dictionary(28586 unique tokens: ['product', 'came', 'exactly', 'as', 'described']...)


In [15]:
def myCorpus(catFilter = None, asin = None, concatenate = False):
    for review in myReviews(catFilter,asin,concatenate):
    # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(review)

In [16]:
import os
tfidfFileName ='../../download/word2vec/reviews_Cell_Phones_and_Accessories_w_Cat_corpus.tfidf'
if os.path.isfile(tfidfFileName)==False:
    tfidf = models.TfidfModel(myCorpus("Cell Phones"))
    tfidf.save(tfidfFileName)
else:
    tfidf = models.TfidfModel.load(tfidfFileName)

print(tfidf)

TfidfModel(num_docs=69127, num_nnz=4450875)


In [ ]:
import pickle

tfidf_apple_5s_FileName ='../../download/word2vec/apple_5s_review.tfidf'
if os.path.isfile(tfidf_apple_5s_FileName)==False:
    apple_5s_corpus = next(myCorpus(asin="B00F3J4B5S",concatenate=True))
    apple_5s_corpus_tfidf = tfidf[apple_5s_corpus]  
    pickle.dump(apple_5s_corpus_tfidf, open(tfidf_apple_5s_FileName, 'wb'))
else:
    apple_5s_corpus_tfidf = pickle.load( open( tfidf_apple_5s_FileName, "rb" ) )
    
    


In [35]:
print(apple_5s_corpus_tfidf)

[(0, 0.05312718552040513), (1, 0.022409272214209717), (2, 0.014606084853337224), (3, 0.0905729849980898), (4, 0.015992619793642745), (5, 0.0943819818876886), (6, 0.052169164530376545), (7, 0.02364408332665259), (8, 0.02097550575298501), (9, 0.058032449438655306), (10, 0.07208593892122526), (11, 0.1334572405879639), (12, 0.04566300749339527), (13, 0.0042301087985577475), (14, 0.025788537379499092), (15, 0.012596971237638106), (16, 0.011245187699306476), (17, 0.03590855162192841), (18, 0.11550580570889703), (19, 0.06000710957795927), (20, 0.08678170846819648), (21, 0.0025392981578692066), (22, 0.0042449668646825965), (23, 0.006560065188170831), (24, 0.1142443767146562), (25, 0.11648178437815244), (26, 0.04907561783032843), (28, 0.07349565537240883), (29, 0.0835472825716422), (30, 0.04813507224479658), (31, 0.014235453563438409), (32, 0.1309326449831593), (33, 0.038829391567855934), (34, 0.013018987055550628), (35, 0.0136657052715009), (36, 0.0657025796668591), (38, 0.009550549058439628),

In [33]:
print(tfidf)

TfidfModel(num_docs=69127, num_nnz=4450875)


In [32]:
print(next(apple_5s_corpus))

StopIteration: 